In [1]:
import os, os.path as op
import shutil
import yaml
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable

import pandas as pd
import xarray as xr
from wand.image import Image as WImage

## Defining the home and data directories
tutorial_home_dir = os.path.abspath(os.getcwd())
work_dir = os.path.abspath(os.getcwd())
print("Tutorial directory: ", tutorial_home_dir)
print("Work directory: ", work_dir)

# Verifying if RAiDER is installed correctly
try:
    import RAiDER
except:
    raise Exception('RAiDER is missing from your PYTHONPATH')

os.chdir(work_dir)

Tutorial directory:  /Users/buzzanga/Software_InSAR/RAiDER-docs_git/notebooks/RAiDER_options
Work directory:  /Users/buzzanga/Software_InSAR/RAiDER-docs_git/notebooks/RAiDER_options


In [128]:
if not op.exists(op.join(work_dir, 'data')):
    !unzip -a data.zip

Archive:  data.zip
   creating: data/
  inflating: data/SRTM_3arcsec_LA.dem.vrt  [text]  
  inflating: data/lat_HR.geo         [binary]
  inflating: data/SRTM_3arcsec_LA.dem  [binary]
  inflating: data/lon_HR.geo         [binary]
  inflating: data/sample_gnss_list.csv  [text]  
  inflating: data/SRTM_3arcsec_LA.hdr  [text]  
  inflating: data/S1-GUNW-D-R-071-tops-20200130_20200124-135156-34956N_32979N-PP-913f-v2_0_4.nc  [binary]


### Helpers

In [2]:
def new_yaml_group(example_yaml, dct_group, dst='tmp.yaml'):
    """ Write a temporary yaml file with the new 'value' for 'key', preserving parms in example_yaml"""
    with open(example_yaml, 'r') as f:
        try:
            params = yaml.safe_load(f)
        except yaml.YAMLError as exc:
            print(exc)
            raise ValueError(f'Something is wrong with the yaml file {example_yaml}')
    
    params = {**params, **dct_group}
    dst = op.join(os.path.dirname(example_yaml), dst)
#     print (params)
    
    with open(dst, 'w') as fh:
        yaml.dump(params, fh, default_flow_style=False)
    
    print ('Wrote new cfg file:', dst)
    return dst

In [3]:
yaml_base = os.path.join(tutorial_home_dir, 'example_yamls', 'raider_example_LA_bbox.yaml')

In [91]:
# check the base file works
!raider.py {yaml_base}

Starting to run the weather model calculation
Time: 20200103
Beginning weather model pre-processing
Extent of the weather model is (xmin, ymin, xmax, ymax):-123.75, 27.50, -111.25, 39.50
Extent of the input is (xmin, ymin, xmax, ymax): -123.00, 28.00, -112.00, 39.00
Output SNWE: [28.0, 39.02, -123.0, -111.98]
Output cube spacing: 0.02
Wrote delays to: /Users/buzzanga/Software_InSAR/RAiDER-docs_git/notebooks/RAiDER_options/GMAO_tropo_20200103T000000_ztd.nc


## Models

In [68]:
from RAiDER.models.allowed import ALLOWED_MODELS
models = ALLOWED_MODELS.copy()
models.remove('NCMR')
print ('Testing Models:', models)

Testing Models ['ERA5', 'ERA5T', 'HRRR', 'GMAO', 'HRES']


In [84]:
## may want to test downloading
shutil.rmtree('weather_files')

In [85]:
for model in models:
    grp  = {'weather_model': model}
    cfg  = new_yaml_group(yaml_base, grp)
    print ('Running:', model)
    !raider.py {cfg} 
    

Wrote new cfg file: /Users/buzzanga/Software_InSAR/RAiDER-docs_git/notebooks/RAiDER_tutorial/example_yamls/tmp.yaml
Running: ERA5
Starting to run the weather model calculation
Time type: <class 'datetime.datetime'>
Time: 20200103
Max integration height is 15000.0 m
Beginning weather model pre-processing
Weather model ERA-5 is available from 1950-01-01 00:00:00-Present
/Users/buzzanga/Miniconda3/envs/RAiDER/lib/python3.10/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'cds.climate.copernicus.eu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
2022-11-29 18:30:52,889 INFO Welcome to the CDS
2022-11-29 18:30:52,890 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-complete
/Users/buzzanga/Miniconda3/envs/RAiDER/lib/python3.10/site-packages/urllib3/connectionpool.py:1045: Insecure

## Date Options

In [92]:
# date start only is default

In [93]:
# date start and end
grp = {'date_group': {'date_start' : 20200103, 'date_end': 20200107}}
cfg = new_yaml_group(yaml_base, grp)
!raider.py {cfg}

Wrote new cfg file: /Users/buzzanga/Software_InSAR/RAiDER-docs_git/notebooks/RAiDER_options/example_yamls/tmp.yaml
Starting to run the weather model calculation
Time: 20200103
Beginning weather model pre-processing
Extent of the weather model is (xmin, ymin, xmax, ymax):-123.75, 27.50, -111.25, 39.50
Extent of the input is (xmin, ymin, xmax, ymax): -123.00, 28.00, -112.00, 39.00
Output SNWE: [28.0, 39.02, -123.0, -111.98]
Output cube spacing: 0.02
Wrote delays to: /Users/buzzanga/Software_InSAR/RAiDER-docs_git/notebooks/RAiDER_options/GMAO_tropo_20200103T000000_ztd.nc
Starting to run the weather model calculation
Time: 20200104
Beginning weather model pre-processing
Extent of the weather model is (xmin, ymin, xmax, ymax):-123.75, 27.50, -111.25, 39.50
Extent of the input is (xmin, ymin, xmax, ymax): -123.00, 28.00, -112.00, 39.00
Output SNWE: [28.0, 39.02, -123.0, -111.98]
Output cube spacing: 0.02
Wrote delays to: /Users/buzzanga/Software_InSAR/RAiDER-docs_git/notebooks/RAiDER_options

In [94]:
# date start and end every 2 days
grp = {'date_group': {'date_start' : 20200103, 'date_end': 20200107, 'date_step': 2}}
cfg = new_yaml_group(yaml_base, grp)
!raider.py {cfg}

Wrote new cfg file: /Users/buzzanga/Software_InSAR/RAiDER-docs_git/notebooks/RAiDER_options/example_yamls/tmp.yaml
Starting to run the weather model calculation
Time: 20200103
Beginning weather model pre-processing
Extent of the weather model is (xmin, ymin, xmax, ymax):-123.75, 27.50, -111.25, 39.50
Extent of the input is (xmin, ymin, xmax, ymax): -123.00, 28.00, -112.00, 39.00
Output SNWE: [28.0, 39.02, -123.0, -111.98]
Output cube spacing: 0.02
Wrote delays to: /Users/buzzanga/Software_InSAR/RAiDER-docs_git/notebooks/RAiDER_options/GMAO_tropo_20200103T000000_ztd.nc
Starting to run the weather model calculation
Time: 20200105
Beginning weather model pre-processing
Extent of the weather model is (xmin, ymin, xmax, ymax):-123.75, 27.50, -111.25, 39.50
Extent of the input is (xmin, ymin, xmax, ymax): -123.00, 28.00, -112.00, 39.00
Output SNWE: [28.0, 39.02, -123.0, -111.98]
Output cube spacing: 0.02
Wrote delays to: /Users/buzzanga/Software_InSAR/RAiDER-docs_git/notebooks/RAiDER_options

In [95]:
# list of dates
grp = {'date_group': {'date_list': [20200110, 20200115]}}
cfg = new_yaml_group(yaml_base, grp)
!raider.py {cfg}

Wrote new cfg file: /Users/buzzanga/Software_InSAR/RAiDER-docs_git/notebooks/RAiDER_options/example_yamls/tmp.yaml
Starting to run the weather model calculation
Time: 20200110
Beginning weather model pre-processing
Extent of the weather model is (xmin, ymin, xmax, ymax):-123.75, 27.50, -111.25, 39.50
Extent of the input is (xmin, ymin, xmax, ymax): -123.00, 28.00, -112.00, 39.00
Output SNWE: [28.0, 39.02, -123.0, -111.98]
Output cube spacing: 0.02
Wrote delays to: /Users/buzzanga/Software_InSAR/RAiDER-docs_git/notebooks/RAiDER_options/GMAO_tropo_20200110T000000_ztd.nc
Starting to run the weather model calculation
Time: 20200115
Beginning weather model pre-processing
Extent of the weather model is (xmin, ymin, xmax, ymax):-123.75, 27.50, -111.25, 39.50
Extent of the input is (xmin, ymin, xmax, ymax): -123.00, 28.00, -112.00, 39.00
Output SNWE: [28.0, 39.02, -123.0, -111.98]
Output cube spacing: 0.02
Wrote delays to: /Users/buzzanga/Software_InSAR/RAiDER-docs_git/notebooks/RAiDER_options

## Time Options

In [96]:
# start time only is default

In [97]:
# start and end time
grp = {'time_group': {'time': '00:00:00', 'end_time': '06:00:00'}}
cfg = new_yaml_group(yaml_base, grp)
!raider.py {cfg}

Wrote new cfg file: /Users/buzzanga/Software_InSAR/RAiDER-docs_git/notebooks/RAiDER_options/example_yamls/tmp.yaml
Starting to run the weather model calculation
Time: 20200103
Beginning weather model pre-processing
Extent of the weather model is (xmin, ymin, xmax, ymax):-123.75, 27.50, -111.25, 39.50
Extent of the input is (xmin, ymin, xmax, ymax): -123.00, 28.00, -112.00, 39.00
Output SNWE: [28.0, 39.02, -123.0, -111.98]
Output cube spacing: 0.02
Wrote delays to: /Users/buzzanga/Software_InSAR/RAiDER-docs_git/notebooks/RAiDER_options/GMAO_tropo_20200103T000000_ztd.nc


## AOI Options

In [135]:
# bbox 
!raider.py {yaml_base}

Starting to run the weather model calculation
Time: 20200103
Beginning weather model pre-processing
Extent of the weather model is (xmin, ymin, xmax, ymax):-123.75, 27.50, -111.25, 39.50
Extent of the input is (xmin, ymin, xmax, ymax): -123.00, 28.00, -112.00, 39.00
Output SNWE: [28.0, 39.02, -123.0, -111.98]
Output cube spacing: 0.02
Wrote delays to: /Users/buzzanga/Software_InSAR/RAiDER-docs_git/notebooks/RAiDER_options/GMAO_tropo_20200103T000000_ztd.nc


In [99]:
##TODO: Change to ++process

# bbox global for GNSS
grp  = {'aoi_group': {'bounding_box': '-90 90 -180 180'}}
cfg  = new_yaml_group(yaml_base, grp)

# !raider.py {cfg} --dostep download_gnss

Wrote new cfg file: /Users/buzzanga/Software_InSAR/RAiDER-docs_git/notebooks/RAiDER_options/example_yamls/tmp.yaml


In [136]:
# geocoded file (GUNW)
prod = 'S1-GUNW-D-R-071-tops-20200130_20200124-135156-34956N_32979N-PP-913f-v2_0_4.nc'
grp  = {'aoi_group': {'geocoded_file': op.join(work_dir, 'data', prod)}}
cfg  = new_yaml_group(yaml_base, grp)
!raider.py {cfg}

Wrote new cfg file: /Users/buzzanga/Software_InSAR/RAiDER-docs_git/notebooks/RAiDER_options/example_yamls/tmp.yaml
Starting to run the weather model calculation
Time: 20200103
Beginning weather model pre-processing
Extent of the weather model is (xmin, ymin, xmax, ymax):-123.75, 27.50, -111.25, 39.50
Extent of the input is (xmin, ymin, xmax, ymax): -119.36, 32.98, -116.24, 34.96
Output SNWE: [32.980000000000004, 34.980000000000004, -119.38, -116.22]
Output cube spacing: 0.02
/Users/buzzanga/Miniconda3/envs/RAiDER/lib/python3.10/site-packages/scipy/interpolate/_rgi.py:358: RuntimeWarning: invalid value encountered in add
  values += np.asarray(self.values[edge_indices]) * weight[vslice]
/Users/buzzanga/Miniconda3/envs/RAiDER/lib/python3.10/site-packages/rasterio/__init__.py:314: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = writer(
Wrote: /Users/buzzanga/Software_InSAR/RAiDER-docs_git/notebooks/RAiDER_options/GMAO_

In [139]:
# geocoded file (SRTM DEM)
dem  = op.join(work_dir, 'data', 'SRTM_3arcsec_LA.dem')

grp  = {'aoi_group': {'geocoded_file': dem},
       }
cfg  = new_yaml_group(yaml_base, grp)
!raider.py {cfg}

Wrote new cfg file: /Users/buzzanga/Software_InSAR/RAiDER-docs_git/notebooks/RAiDER_options/example_yamls/tmp.yaml
Using user DEM: SRTM_3ARCSEC_LA.DEM
Starting to run the weather model calculation
Time: 20200103
Beginning weather model pre-processing
Extent of the weather model is (xmin, ymin, xmax, ymax):-123.75, 27.50, -111.25, 39.50
Extent of the input is (xmin, ymin, xmax, ymax): -123.00, 29.00, -113.00, 39.00
Output SNWE: [29.0, 39.02, -123.0, -112.98]
Output cube spacing: 0.02
/Users/buzzanga/Miniconda3/envs/RAiDER/lib/python3.10/site-packages/scipy/interpolate/_rgi.py:358: RuntimeWarning: invalid value encountered in add
  values += np.asarray(self.values[edge_indices]) * weight[vslice]
/Users/buzzanga/Miniconda3/envs/RAiDER/lib/python3.10/site-packages/rasterio/__init__.py:314: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = writer(
Wrote: /Users/buzzanga/Software_InSAR/RAiDER-docs_git/notebooks/RAiDER_optio

In [140]:
# ISCE lat/lon files (Hampton Roads)
grp = {'aoi_group': {'lat_file': f'{work_dir}/data/lat_HR.geo', 'lon_file': f'{work_dir}/data/lon_HR.geo'}, 
       'weather_model': 'ERA5',
      'date_group': {'date_start': '20181113'},
      'time_group': {'time': '23:00:00'}
      }

cfg = new_yaml_group(yaml_base, grp)
!raider.py {cfg}

Wrote new cfg file: /Users/buzzanga/Software_InSAR/RAiDER-docs_git/notebooks/RAiDER_options/example_yamls/tmp.yaml
/Users/buzzanga/Miniconda3/envs/RAiDER/lib/python3.10/site-packages/rasterio/__init__.py:304: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)
Starting to run the weather model calculation
Time: 20181113
Beginning weather model pre-processing
Extent of the weather model is (xmin, ymin, xmax, ymax):-77.06, 36.27, -75.55, 37.52
Extent of the input is (xmin, ymin, xmax, ymax): -76.65, 36.67, -75.93, 37.31
Output SNWE: [36.660000000000004, 37.32, -76.66, -75.9]
Output cube spacing: 0.02
/Users/buzzanga/Miniconda3/envs/RAiDER/lib/python3.10/site-packages/rasterio/__init__.py:304: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = DatasetReader(path, driver=driver, sharing=sharing, 

In [117]:
# gnss station file
yaml_gnss = yaml_base.replace('bbox', 'GNSS')

!raider.py {yaml_gnss}

Starting to run the weather model calculation
Time: 20200103
Beginning weather model pre-processing
Extent of the weather model is (xmin, ymin, xmax, ymax):-123.75, 27.50, -111.25, 39.50
Extent of the input is (xmin, ymin, xmax, ymax): -122.96, 29.03, -113.01, 39.00
Output SNWE: [29.02, 39.02, -122.96000000000001, -113.0]
Output cube spacing: 0.02
Wrote delays to: /Users/buzzanga/Software_InSAR/RAiDER-docs_git/notebooks/RAiDER_options/GMAO_wet_20200103T000000_ztd.GTiff


## Height Options

In [118]:
# list of heights
!raider.py {yaml_base}

Starting to run the weather model calculation
Time: 20200103
Beginning weather model pre-processing
Extent of the weather model is (xmin, ymin, xmax, ymax):-123.75, 27.50, -111.25, 39.50
Extent of the input is (xmin, ymin, xmax, ymax): -123.00, 28.00, -112.00, 39.00
Output SNWE: [28.0, 39.02, -123.0, -111.98]
Output cube spacing: 0.02
Wrote delays to: /Users/buzzanga/Software_InSAR/RAiDER-docs_git/notebooks/RAiDER_options/GMAO_tropo_20200103T000000_ztd.nc


In [141]:
# dem
# geocoded file (SRTM DEM)
grp  = {'height_group': {'dem': f'{work_dir}/data/SRTM_3arcsec_LA.dem', 'use_dem_latlon': True}}
cfg  = new_yaml_group(yaml_base, grp)

!raider.py {cfg}

Wrote new cfg file: /Users/buzzanga/Software_InSAR/RAiDER-docs_git/notebooks/RAiDER_options/example_yamls/tmp.yaml
Starting to run the weather model calculation
Time: 20200103
Beginning weather model pre-processing
Extent of the weather model is (xmin, ymin, xmax, ymax):-123.75, 27.50, -111.25, 39.50
Extent of the input is (xmin, ymin, xmax, ymax): -123.00, 29.00, -113.00, 39.00
Output SNWE: [29.0, 39.02, -123.0, -112.98]
Output cube spacing: 0.02
/Users/buzzanga/Miniconda3/envs/RAiDER/lib/python3.10/site-packages/scipy/interpolate/_rgi.py:358: RuntimeWarning: invalid value encountered in multiply
  values += np.asarray(self.values[edge_indices]) * weight[vslice]
/Users/buzzanga/Miniconda3/envs/RAiDER/lib/python3.10/site-packages/rasterio/__init__.py:314: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = writer(
Wrote: /Users/buzzanga/Software_InSAR/RAiDER-docs_git/notebooks/RAiDER_options/GMAO_wet_20200103T000000_ztd

In [145]:
# ISCE lat/lon files (Hampton Roads)
grp = {'aoi_group': {'lat_file': f'{work_dir}/data/lat_HR.geo', 'lon_file': f'{work_dir}/data/lon_HR.geo'}, 
       'height_group': {'dem': f'{work_dir}/data/SRTM_3arcsec_LA.dem', 'use_dem_latlon': True},
       'weather_model': 'ERA5',
       'date_group': {'date_start': '20181113'},
       'time_group': {'time': '23:00:00'}
      }

cfg = new_yaml_group(yaml_base, grp)
!raider.py {cfg}

Wrote new cfg file: /Users/buzzanga/Software_InSAR/RAiDER-docs_git/notebooks/RAiDER_options/example_yamls/tmp.yaml
Starting to run the weather model calculation
Time: 20181113
Beginning weather model pre-processing
Extent of the weather model is (xmin, ymin, xmax, ymax):-77.06, 36.27, -75.55, 37.52
Extent of the input is (xmin, ymin, xmax, ymax): -123.00, 29.00, -113.00, 39.00
Number of weather model nodes: 1344
Shape of weather model: (6, 7, 32)
Bounds of the weather model: 36.27/37.52/-77.06/-75.55 (SNWE)
Weather model: ERA-5
Mean value of the wet refractivity: 20.226200
Mean value of the hydrostatic refractivity: 138.367264

======Weather Model class object=====
Weather model time: 2018-11-13 23:00:00
Latitude resolution: 0.2
Longitude resolution: 0.2
Native projection: epsg:4326
ZMIN: -100.0
ZMAX: 15000.0
k1 = 0.776
k2 = 0.233
k3 = 3750.0
Humidity type = q
Class name: ea
Dataset: era5
A: []
B: []
Number of points in Lon/Lat = 6/7
Total number of grid points (3D): 1344

Output SNWE:

## LOS Options

In [146]:
yaml_orbit = op.join(work_dir, 'example_yamls', 'raider_example_UK-isce.yaml')
orbit_file = op.join(work_dir, 'orbits', 'S1B_OPER_AUX_POEORB_OPOD_20211122T112354_V20211101T225942_20211103T005942.EOF')

In [147]:
# orbit file, bounding box, height levels, ray_trace
!raider.py {yaml_orbit}

Starting to run the weather model calculation
Time: 20211102
Beginning weather model pre-processing
Extent of the weather model is (xmin, ymin, xmax, ymax):-5.40, 51.60, 0.35, 57.35
Extent of the input is (xmin, ymin, xmax, ymax): -5.00, 52.00, 0.00, 57.00
Output SNWE: [52.980000000000004, 56.0, -4.0, -0.98]
Output cube spacing: 0.02
Look direction: right
Processing slice 1 / 5: 0.0
Processing slice 2 / 5: 50.0
Processing slice 3 / 5: 100.0
Processing slice 4 / 5: 500.0
Processing slice 5 / 5: 1000.0
Wrote delays to: /Users/buzzanga/Software_InSAR/RAiDER-docs_git/notebooks/RAiDER_options/ERA5_tropo_20211102T062159_ray.nc


In [149]:
# orbit file, bounding box, height levels, SLANT
grp  = {'los_group': {'ray_trace': False, 
                      'orbit_file': orbit_file}}
cfg  = new_yaml_group(yaml_orbit, grp)
!raider.py {cfg}

Wrote new cfg file: /Users/buzzanga/Software_InSAR/RAiDER-docs_git/notebooks/RAiDER_options/example_yamls/tmp.yaml
Starting to run the weather model calculation
Time: 20211102
Beginning weather model pre-processing
Extent of the weather model is (xmin, ymin, xmax, ymax):-5.40, 51.60, 0.35, 57.35
Extent of the input is (xmin, ymin, xmax, ymax): -4.00, 53.00, -1.00, 56.00
Output SNWE: [52.980000000000004, 56.0, -4.0, -0.98]
Output cube spacing: 0.02
Wrote delays to: /Users/buzzanga/Software_InSAR/RAiDER-docs_git/notebooks/RAiDER_options/ERA5_tropo_20211102T062159_std.nc


In [150]:
# orbit file, bounding box, NO height levels (so at weather model)
grp  = {'height_group': {}}
cfg  = new_yaml_group(yaml_orbit, grp)
!raider.py {cfg}

Wrote new cfg file: /Users/buzzanga/Software_InSAR/RAiDER-docs_git/notebooks/RAiDER_options/example_yamls/tmp.yaml
Starting to run the weather model calculation
Time: 20211102
Beginning weather model pre-processing
Extent of the weather model is (xmin, ymin, xmax, ymax):-5.40, 51.60, 0.35, 57.35
Extent of the input is (xmin, ymin, xmax, ymax): -5.00, 52.00, 0.00, 57.00
Output SNWE: [52.980000000000004, 56.0, -4.0, -0.98]
Output cube spacing: 0.02
Look direction: right
Processing slice 1 / 32: -500.0
Processing slice 2 / 32: -300.0
Processing slice 3 / 32: -200.0
Processing slice 4 / 32: -100.0
Processing slice 5 / 32: -50.0
Processing slice 6 / 32: -20.0
Processing slice 7 / 32: -10.0
Processing slice 8 / 32: 106.54000091552734
Processing slice 9 / 32: 566.5399780273438
Processing slice 10 / 32: 798.719970703125
Processing slice 11 / 32: 987.1500244140625
Processing slice 12 / 32: 1459.9100341796875
Processing slice 13 / 32: 1910.760009765625
Processing slice 14 / 32: 3089.25
Processin

In [ ]:
#### TO DO

In [153]:
# orbit file, lat/lon grid
orbit_file_HR = op.join(work_dir, 'orbits', 'S1A_OPER_AUX_POEORB_OPOD_20181203T120749_V20181112T225942_20181114T005942.EOF')

grp = {'aoi_group': {'lat_file': f'{work_dir}/data/lat_HR.geo', 'lon_file': f'{work_dir}/data/lon_HR.geo'}, 
       'weather_model': 'ERA5',
      'date_group': {'date_start': '20181113'},
      'time_group': {'time': '23:00:00'},
      'los_group':  {'ray_trace': False, 
                      'orbit_file': orbit_file_HR}
      }

cfg = new_yaml_group(yaml_base, grp)
!raider.py {cfg}

Wrote new cfg file: /Users/buzzanga/Software_InSAR/RAiDER-docs_git/notebooks/RAiDER_options/example_yamls/tmp.yaml
/Users/buzzanga/Miniconda3/envs/RAiDER/lib/python3.10/site-packages/rasterio/__init__.py:304: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)
Starting to run the weather model calculation
Time: 20181113
Beginning weather model pre-processing
Extent of the weather model is (xmin, ymin, xmax, ymax):-77.06, 36.27, -75.55, 37.52
Extent of the input is (xmin, ymin, xmax, ymax): -76.65, 36.67, -75.93, 37.31
Output SNWE: [36.660000000000004, 37.32, -76.66, -75.9]
Output cube spacing: 0.02
/Users/buzzanga/Miniconda3/envs/RAiDER/lib/python3.10/site-packages/rasterio/__init__.py:304: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = DatasetReader(path, driver=driver, sharing=sharing, 

## Projection Options

In [41]:
# os.chdir('UTM_DEM')

In [42]:
## UTM DEM with WGS Model
# !raider.py raiderDelay_utm_CA-ERA5.yaml

In [43]:
## UTM DEM with UTM Model
# !raider.py raiderDelay_utm_CA-HRRR.yaml

In [44]:
## WGS DEM with UTM Model
# !raider.py raiderDelay_geo_CA-HRRR.yaml

In [45]:
# os.chdir('..')